# i) Regra da soma

In [55]:
import os
import numpy as np
import pandas as pd
import clustering

## Datasets

In [56]:
DATA_BASE_PATH = "./data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar")

fac = np.loadtxt(FAC_FILE, dtype=int)
fou = np.loadtxt(FOU_FILE, dtype=float)
kar = np.loadtxt(KAR_FILE, dtype=float)

## Importando melhor resultado

In [57]:
best_result = clustering.import_best_result("data/melhor_resultado_todas.pickle")
partition, _ = clustering.get_hard_patitions(best_result["membership_degree"])


## Probabilidade à priori das classes

In [58]:
Pw = np.array([len(c)/2000 for c in partition])
Pw

(array([0.1175, 0.116 , 0.1355, 0.1465, 0.0405, 0.01  , 0.1305, 0.151 ,
        0.0815, 0.071 ]),
 1.0)

## Função de densidade

In [61]:
def calc_gaussian_density_prob(xk, d, means, var, cov_matrix):
    coef = np.power(2*np.pi, -d/2) 
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
    sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
    diff = xk - means
    exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
    exp_func = np.exp(exp_exp)
    
    return coef * sqrt_det_inv_cov * exp_func

In [62]:
def calc_gaussian_bayesian_data(x, partition):
    n,d = x.shape
    qtd_w = len(partition)
    means = np.array([x[idxs].mean(axis=0) for idxs in partition])
    var = np.array([((x[idxs]-means[i])**2).mean(axis=0) for i, idxs in enumerate(partition)])
    cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

    for i in range(qtd_w):
        np.fill_diagonal(cov_matrix[i], var[i])
                
    p_x_w = np.empty((n, qtd_w))
    
    for i in range(qtd_w):
        for k in range(n):
            p_x_w[k, i] = calc_gaussian_density_prob(x[k], d, means[i], var[i], cov_matrix[i])   
    
    return p_x_w

def calc_prob_posteriori(p_x_w, Pw):
    p_w_x = np.empty((10, 2000))
    
    for i in range(10):
        for k in range(2000):
            sum_all = np.dot(p_x_w[k], Pw)
            p_w_x[i,k] = (p_x_w[k, i] * Pw[i])/sum_all
    
    y_pred = p_w_x.argmax(axis = 0)
    
    return y_pred, p_w_x


## Densidades por dataset

In [63]:
fac_byn_density_probs = calc_gaussian_bayesian_data(fac, partition)
fou_byn_density_probs = calc_gaussian_bayesian_data(fou, partition)
kar_byn_density_probs = calc_gaussian_bayesian_data(kar, partition)

In [64]:
fac_byn_density_probs[0,0], fou_byn_density_probs[0,0], kar_byn_density_probs[0,0]

(0.0, 3.243154665015897e+60, 7.254547591145725e-49)

## Prob. à priori por view

In [65]:
fac_byn_y_pred, fac_byn_posteriori_probs = calc_prob_posteriori(fac_byn_density_probs, Pw)
fou_byn_y_pred, fou_byn_posteriori_probs = calc_prob_posteriori(fou_byn_density_probs, Pw)
kar_byn_y_pred, kar_byn_posteriori_probs = calc_prob_posteriori(kar_byn_density_probs, Pw)


/home/thiago/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


## Regra da soma

Precisei tirar o fac porque seus valores são nulos. TENTAR CORRIGIR ISSO

In [66]:
def regra_soma_padrao(fac_p_w_x, fou_p_w_x, kar_p_w_x, Pw):
    x_sum_w = np.empty((10, 2000))
    
    for i in range(10):
        for k in range(2000):
            views_sum =  fou_p_w_x[i,k] + kar_p_w_x[i,k]
            # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
            x_sum_w[i,k] = (1-2)*Pw[i] + views_sum
            
    y_pred = x_sum_w.argmax(axis = 0) 
    return y_pred

## Esitmador do scikit

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class RegraSomaClassificaforBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition):
         self.partition = partition
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        return self
    
    def calc_gaussian_bayesian_data(x, partition):
        n,d = x.shape
        qtd_w = len(partition)
        self.means = np.array([x[idxs].mean(axis=0) for idxs in partition])
        self.var = np.array([((x[idxs]-self.means[i])**2).mean(axis=0) for i, idxs in enumerate(partition)])
        self.cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

        for i in range(qtd_w):
            np.fill_diagonal(self.cov_matrix[i], self.var[i])

        p_x_w = np.empty((n, qtd_w))

        for i in range(qtd_w):
            for k in range(n):
                p_x_w[k, i] = calc_gaussian_density_prob(x[k], 
                                                         d, 
                                                         self.means[i], 
                                                         self.var[i], 
                                                         self.cov_matrix[i])   

        return p_x_w

     def predict(self, X):

        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)
        closest = np.argmin(euclidean_distances(X, self.X_), axis=1)
        return self.y_[closest]

In [67]:
y_byn_pred_all = regra_soma_padrao(fac_byn_posteriori_probs, 
                                    fou_byn_posteriori_probs, 
                                    kar_byn_posteriori_probs, Pw)

In [68]:
from sklearn.metrics import accuracy_score, f1_score

y_true = clustering.get_instances_class()

print("Acurácia: ", accuracy_score(y_true, y_byn_pred_all))
print("Medida-F: ", f1_score(y_true, y_byn_pred_all, average="macro"))

Acurácia:  0.671
Medida-F:  0.6514961439404549


# K-Vizinhos

## Normalizando 


In [69]:
from sklearn.preprocessing import minmax_scale

fac_norm = minmax_scale(fac)
fou_norm = minmax_scale(fou)
kar_norm = minmax_scale(kar)

## Distâncias entre os elementos

In [70]:
from sklearn.metrics.pairwise import euclidean_distances

fac_dist = euclidean_distances(fac_norm, fac_norm)
fou_dist = euclidean_distances(fou_norm, fou_norm)
kar_dist = euclidean_distances(kar_norm, kar_norm)

## Função de densidade

In [71]:
def calc_knn_density_prob(view_dists, k, Pw):
    p_x_w = np.empty((2000, 10))
    k_vizinhos = np.empty((2000, k))
    kvizihos_temp = view_dists.argsort(axis=1)[:,:k+1]
    
    for i in range(2000):
        k_vizinhos[i,:] = np.delete(kvizihos_temp[i], 0)
    
    del kvizihos_temp
    
    for i in range(10):
        for j in range(2000):
            #p_w_x[i,j] = (((k_vizinhos[j] == i).sum()/len(partition[i]))/V)*Pw[i]
            p_x_w[j,i] = (k_vizinhos[j] == i).sum()/k
                
    #y_pred = p_x_w.argmax(axis = 0)
    y_pred = k_vizinhos.argmax(axis = 0)
    
    return y_pred, p_x_w
    

## Desidades por dataset

In [72]:
_, fac_knn_density_probs = calc_knn_density_prob(fac_dist, 100, Pw)
_, fou_knn_density_probs = calc_knn_density_prob(fou_dist, 100, Pw)
_, kar_knn_density_probs = calc_knn_density_prob(kar_dist, 100, Pw)

## Prob. à priori por view

In [73]:
fac_knn_y_pred, fac_knn_posteriori_probs = calc_prob_posteriori(fac_knn_density_probs, Pw)
fou_knn_y_pred, fou_knn_posteriori_probs = calc_prob_posteriori(fou_knn_density_probs, Pw)
kar_knn_y_pred, kar_knn_posteriori_probs = calc_prob_posteriori(kar_knn_density_probs, Pw)

/home/thiago/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


## Regra da soma

In [84]:
def regra_soma_knn(fac_p_w_x, fou_p_w_x, kar_p_w_x, Pw):
    x_sum_w = np.empty((10, 2000))
    
    for i in range(10):
        for k in range(2000):
            #views_sum =  fou_p_w_x[i,k] + kar_p_w_x[i,k]
            views_sum =  fac_p_w_x[i,k] + fou_p_w_x[i,k] + kar_p_w_x[i,k]
            x_sum_w[i,k] = (1-2)*Pw[i] + views_sum
            
    y_pred = x_sum_w.argmax(axis = 0) 
    return y_pred

In [85]:
y_pred_knn_all = regra_soma_knn(fac_knn_posteriori_probs, 
                                   fou_knn_posteriori_probs, 
                                   kar_knn_posteriori_probs, Pw)

In [86]:
print("Acurácia: ", accuracy_score(y_true, y_pred_knn_all))
print("Medida-F: ", f1_score(y_true, y_pred_knn_all, average="macro"))

Acurácia:  0.01
Medida-F:  0.001984126984126984


# Janela de Parzen

## Função de densidade

In [77]:
# def parzen_density_function(view, h, partition):
#     p_x_w = np.empty((10, 2000))
#     dims = view.shape[1]
#     elements_in_window = np.empty((10, dims))
    
#     def K(x):
#         return np.exp(-x**2/2)/np.sqrt(2*np.pi)

#     for i in range(10):
#         n = len(partition[i])
#         for k in range(2000):
#             p = np.array([np.array([K((view[k,j] - view[e,j])/h) for j in range(dims)]).prod() 
#                  for e in partition[i]]).sum()
            
#             p_x_w[i,k] = (1/(n*h**dims))
            
#     return p_x_w

def parzen_density_function(view, h, partition):
    p_x_w = np.empty((2000, 10))
    dims = view.shape[1]
    
    for i in range(10):
        n = len(partition[i])
        x_view = view[partition[i],:]
        
        for k in range(2000):
            diff = (view[k] - x_view)/h
            gaussian_diff = np.exp(-diff**2/2)/np.sqrt(2*np.pi)
            prod_dims = gaussian_diff.prod(axis=0)
            p_x_w[k,i] = (1/(n*h**dims)) * prod_dims.sum()
            print(p_x_w[k,i])
            
    return p_x_w

In [78]:
fac_parzen_density_probs = parzen_density_function(fac, 50, partition)
fou_parzen_density_probs = parzen_density_function(fou, 50, partition)
kar_parzen_density_probs = parzen_density_function(kar, 50, partition)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


...

8.231142579106411e-231
8.149151581280133e-231
8.252279099068583e-231
8.113180517180823e-231
7.852000128070644e-231
8.136468286409586e-231
8.292895220230325e-231
8.439992560895457e-231
8.327169589553454e-231
8.59230516674849e-231
7.987290331779518e-231
8.270301140896395e-231
8.073082849184385e-231
8.029654682629752e-231
8.336800256527415e-231
7.809423931979876e-231
8.424883814402393e-231
7.984381595575822e-231
8.291005693746726e-231
7.599121480755551e-231
8.246299783704089e-231
8.01764215703079e-231
8.580877158593682e-231
8.168350793016708e-231
8.549760417036765e-231
8.274682224599857e-231
8.44578443671687e-231
8.150931940040834e-231
8.178854812044813e-231
8.367106125524986e-231
8.439856317541749e-231
8.305420236163047e-231
8.443691747666557e-231
8.369843857851527e-231
8.3772527298057e-231
8.285091359318338e-231
8.51430118021414e-231
8.642639951206064e-231
8.529056181634412e-231
8.644963517647791e-231
8.333408643608747e-231
8.440150210337886e-231
8.572341177093842e-231
8.581787545847145

5.060873463646012e-175
4.8302255614951827e-175
5.0128451830571446e-175
4.946554073904555e-175
5.0137864486784236e-175
4.711359111000659e-175
4.9378439186032863e-175
4.9203272514401945e-175
4.9749577300602005e-175
4.9830678208544893e-175
4.9424588831780167e-175
4.9685704122519876e-175
4.9951127402182104e-175
4.971659664590647e-175
5.024541577889224e-175
5.058623496070756e-175
5.056702475364601e-175
4.746571220152284e-175
4.9406699156542923e-175
4.990519858676348e-175
5.002931345958333e-175
5.102216899286997e-175
5.021242532870097e-175
4.8626072524131173e-175
4.842594394327189e-175
4.951786248214142e-175
4.842603930911716e-175
4.9714184079265066e-175
5.111029189473415e-175
4.951830440501787e-175
4.99599600475099e-175
5.076570102622496e-175
5.032395944133803e-175
4.9070844142316705e-175
5.0012758058479326e-175
5.097446102434422e-175
4.936700451327999e-175
4.83613644429946e-175
4.936823604401448e-175
4.961883501907833e-175
4.954734327142667e-175
4.977651859793927e-175
4.8531505489604454e-1

4.9116401584316097e-175
4.80310660633961e-175
4.847091396556689e-175
4.970355186780418e-175
4.9108028600145516e-175
4.8606377014989034e-175
4.8017159362792905e-175
4.96445388885359e-175
5.037909334585548e-175
4.855617360205466e-175
4.893533281736734e-175
4.8310845198510754e-175
4.9389008203348155e-175
5.0205336305617075e-175
4.807362041996515e-175
4.8348299588236674e-175
4.995735206151703e-175
4.718379599142589e-175
5.019300350287641e-175
4.921679359421867e-175
4.8911889291926605e-175
4.9203668961844367e-175
5.049212773781759e-175
4.82682225336958e-175
4.817203153141303e-175
4.9293391908287226e-175
4.966062732479034e-175
4.8642208470521416e-175
4.982377078463846e-175
4.8834900746213446e-175
5.075004153341542e-175
5.005694069278437e-175
4.912088954605096e-175
4.8755319245814084e-175
4.951872872223044e-175
4.8845108559941697e-175
4.857396578561839e-175
4.938457052664522e-175
5.0194796500523556e-175
4.959709155158978e-175
4.9888836992174295e-175
4.9271296853841086e-175
5.004523299137828e-

1.3996865931858626e-166
1.3553490224963319e-166
1.442192199392059e-166
1.4113529749849144e-166
1.4289082952369753e-166
1.4082769771413725e-166
1.4498532361920056e-166
1.3991187595700433e-166
1.4428577133187556e-166
1.4387097253779162e-166
1.3665023462658045e-166
1.3701510166230955e-166
1.4185226318669514e-166
1.367684331985923e-166
1.4039364325595366e-166
1.4461109204321664e-166
1.4256177463804067e-166
1.3404779622476424e-166
1.3709626320618302e-166
1.4292637023289007e-166
1.401172789382224e-166
1.405947415474828e-166
1.4044173025575107e-166
1.4115231517556833e-166
1.4460682521345598e-166
1.4329003006512217e-166
1.3982906129693907e-166
1.4311208098321648e-166
1.363022031675025e-166
1.3795114102351014e-166
1.383502514280351e-166
1.4306634224595887e-166
1.4171373378563622e-166
1.4031649539652144e-166
1.361453666286853e-166
1.402272359856164e-166
1.4202178216126804e-166
1.3862896849277913e-166
1.43628190746078e-166
1.4071012478383678e-166
1.400383987373011e-166
1.4277117376411337e-166
1.3

1.3784097111035943e-166
1.4050310467411563e-166
1.3917420314376472e-166
1.3931635974345106e-166
1.3953332691205194e-166
1.366917266832206e-166
1.4012378281631183e-166
1.4182163658993085e-166
1.4521634878907223e-166
1.3944421468244688e-166
1.442509130886687e-166
1.361491383811415e-166
1.4071582211076072e-166
1.409525471364543e-166
1.3889504139284144e-166
1.4240668549878937e-166
1.4170851030042624e-166
1.4170160498741796e-166
1.4162312711138532e-166
1.4236536239228067e-166
1.4381178393013152e-166
1.42151274160446e-166
1.407988641644407e-166
1.432637586947623e-166
1.3888941896642982e-166
1.4591848326114774e-166
1.4053201324066786e-166
1.412054720734985e-166
1.4068778060904062e-166
1.431822742439465e-166
1.4170849970316064e-166
1.422736695924273e-166
1.4322522237947092e-166
1.4423967928112307e-166
1.3925362652427598e-166
1.4510186277730147e-166
1.4656145889348336e-166
1.450115273609356e-166
1.4111865987852448e-166
1.4148641911114553e-166
1.4042828515693926e-166
1.4129881753036918e-166
1.41

## Probabilidade a priori

In [79]:
fac_prazen_y_pred, fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
fou_prazen_y_pred, fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
fou_prazen_y_pred, kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

/home/thiago/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


In [80]:
y_prazen_pred_all = regra_soma_padrao(fac_prazen_posteriori_probs, 
                                    fou_prazen_posteriori_probs, 
                                    kar_prazen_posteriori_probs, Pw)

In [81]:
set(y_prazen_pred_all)

{5}

In [82]:
from sklearn.metrics import accuracy_score, f1_score

y_true = clustering.get_instances_class()

print("Acurácia: ", accuracy_score(y_true, y_prazen_pred_all))
print("Medida-F: ", f1_score(y_true, y_prazen_pred_all, average="macro"))

Acurácia:  0.1
Medida-F:  0.01818181818181818
